**Agents for your agents!**

There are two main ways to layer agents together into a multi-agent system:

1. Agents as Tools
2. Agent Routing

We'll discuss each in their own sections below.

In [1]:
# This page will use the following imports:

from lasagna import Model, EventCallback, AgentRun
from lasagna import (
    recursive_extract_messages,
    override_system_prompt,
    flat_messages,
    extraction,
    chained_runs,
    BoundAgentCallable,
)
from lasagna import known_models
from lasagna.tui import tui_input_loop

from pydantic import BaseModel, Field
from enum import Enum

import os

from dotenv import load_dotenv

We need to set up our "binder" (see the [quickstart guide](../quickstart.ipynb) for what this is).

In [2]:
load_dotenv()

if os.environ.get('OPENAI_API_KEY'):
    print('Using OpenAI')
    binder = known_models.openai_gpt_4o_binder

elif os.environ.get('ANTHROPIC_API_KEY'):
    print('Using Anthropic')
    binder = known_models.anthropic_claude_sonnet_4_binder

else:
    assert False, "Neither OPENAI_API_KEY nor ANTHROPIC_API_KEY is set! We need at least one to do this demo."

Using OpenAI


## Agents as Tools

The simplest way to combine agents is to pass one (or more) agents as "tools" to another agent.

Here is a simple example:

In [3]:
async def joke_specialist(
    model: Model,
    event_callback: EventCallback,
    prev_runs: list[AgentRun],
) -> AgentRun:
    """
    Use this agent when the user seems discouraged and needs to feel better.
    This tool will return the perfect joke to cheer the user up.
    """
    messages = recursive_extract_messages(prev_runs, from_tools=False, from_extraction=False)
    messages = override_system_prompt(messages, "You are a joke-telling specialist. You always tell a joke related to the user's most recent message. Your response must contain **only** the joke.")
    new_messages = await model.run(event_callback, messages, tools=[])
    return flat_messages('joke_specialist', new_messages)


async def root_agent(
    model: Model,
    event_callback: EventCallback,
    prev_runs: list[AgentRun],
) -> AgentRun:
    messages = recursive_extract_messages(prev_runs, from_tools=False, from_extraction=False)
    messages = override_system_prompt(messages, 'You are a generic assistant. Use your tools when necessary.')

    new_messages = await model.run(
        event_callback,
        messages,
        tools=[
            joke_specialist,   # <-- 🔨 downstream agent as a tool
        ],
    )

    return flat_messages('root_agent', new_messages)

In [5]:
await tui_input_loop(binder(root_agent))   # type: ignore[top-level-await]

>  Hi!


Hi there! How can I help you today?


>  Oh, I'm sick. :(


joke_specialist({})
Why did the sick computer go to the doctor?  
It had a virus! -> Why did the sick computer go to the doctor?  
It had a virus!
Oh no, I'm sorry you're not feeling well! Here's a little joke to cheer you up:

Why did the sick computer go to the doctor?  
It had a virus!  

Hope that brings at least a little smile. Feel better soon! 😊


>  exit


See the [Agents as Tools](../recipes/agents_as_tools.ipynb) recipe for another example.

## Agent Routing

The most flexible way to combine agents is to have agents delegate to one another ("routing"). The router agent's job is to delegate. It might delegate _wholesale_, or it might _transform_ the prompt before delegating. It might delegate to a _single_ downstream agent, or to _several_ downstream agents. This is what makes it so flexible!

The recipe for routing is to combine _structured output_ with good ol' programming.

Here is an example, extending the example above to be more flexible:

In [7]:
class Mood(Enum):
    happy = 'happy'
    sad = 'sad'
    neutral = 'neutral'


class MessageClassification(BaseModel):
    thoughts: str = Field(description="Your free-form thoughts about the user's most recent message, and what mood the user may be in.")
    mood: Mood = Field(description="Your determination of the user's mood based on their most recent message. If it is not clear, output 'neutral'.")


class RouterAgent:
    def __init__(
        self,
        cheer_up_agent: BoundAgentCallable,
        default_agent: BoundAgentCallable,
    ) -> None:
        self.cheer_up_agent = cheer_up_agent
        self.default_agent = default_agent

    async def __call__(
        self,
        model: Model,
        event_callback: EventCallback,
        prev_runs: list[AgentRun],
    ) -> AgentRun:
        messages = recursive_extract_messages(prev_runs, from_tools=False, from_extraction=False)
        messages = override_system_prompt(messages, "You classify the user's mood.")

        message, result = await model.extract(
            event_callback,
            messages = messages,
            extraction_type = MessageClassification,
        )

        extraction_run = extraction('router_agent', message, result)

        downstream_agent = (self.cheer_up_agent if result.mood == Mood.sad else self.default_agent)

        downstream_run = await downstream_agent(event_callback, prev_runs)

        return chained_runs('router_agent', [extraction_run, downstream_run])

In [8]:
async def joke_telling_agent(
    model: Model,
    event_callback: EventCallback,
    prev_runs: list[AgentRun],
) -> AgentRun:
    messages = recursive_extract_messages(prev_runs, from_tools=False, from_extraction=False)
    messages = override_system_prompt(messages, "You are a joke-telling specialist. You always tell a joke related to the user's most recent message. Cheer the user up by telling a joke!")
    new_messages = await model.run(event_callback, messages, tools=[])
    return flat_messages('joke_telling_agent', new_messages)


async def generic_agent(
    model: Model,
    event_callback: EventCallback,
    prev_runs: list[AgentRun],
) -> AgentRun:
    messages = recursive_extract_messages(prev_runs, from_tools=False, from_extraction=False)
    messages = override_system_prompt(messages, "You are a helpful assistant.")
    new_messages = await model.run(event_callback, messages, tools=[])
    return flat_messages('generic_agent', new_messages)


my_agent = RouterAgent(
    cheer_up_agent = binder(joke_telling_agent),
    default_agent = binder(generic_agent),
)


await tui_input_loop(binder(my_agent))   # type: ignore[top-level-await]

>  Hi!


MessageClassification({"thoughts":"The user greets warmly and positively. This usually reflects a happy or friendly mood.","mood":"happy"})
Hello! 😊 How can I assist you today?


>  Oh, I'm sick. :(


MessageClassification({"thoughts":"The user appears to be feeling unwell and sad based on their message.","mood":"sad"})
Oh no, I'm sorry to hear that! Here's a joke to hopefully nurse your spirits back to health:  

Why did the doctor carry a red pen?  
In case they needed to draw blood!  

Hope you feel better soon! 🩺😊


>  Thanks!


MessageClassification({"thoughts":"The user expressed gratitude but hasn't provided much emotional information in their recent message. Their mood toward me seems polite and appreciative, but overall they may still feel unwell or neutral due to being sick.","mood":"neutral"})
You're welcome! 😊 Rest up, drink plenty of fluids, and take care of yourself. If you need anything else, I'm here to help. Hope you’re feeling like your fabulous self in no time! 💛✨


>  exit


See the [Agent Routing](../recipes/routing_agent.ipynb) recipe for a working example.